In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend
from keras.layers import Embedding
from keras.layers import Concatenate

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot
from math import sqrt
from PIL import Image
import os

from numpy import load
from tensorflow.keras import backend

import shutil
import os
import cv2
import pickle as pkl
import numpy as np
from PIL import Image
import PIL
import numpy as np
import h5py
import scipy.io
import json
import PIL
import matplotlib.image as mpimg
from os import walk, getcwd
import glob
from functools import partial
from numpy import savetxt
from numpy import save

import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from keras.initializers import RandomNormal
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D, Concatenate
from keras.layers import merge
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import Adam, RMSprop
from keras.utils import plot_model
from PIL import Image
from tensorflow.python.framework.ops import disable_eager_execution, enable_eager_execution

# split a dataset into train and test sets
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import splitfolders

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os
import tensorflow_addons as tfa 

## Generate Images

In [2]:
gen_model = tf.keras.models.load_model(r'C:\Users\lasse\Desktop\MainFolder\Final15\pgangenFinal15')
disc_model = tf.keras.models.load_model(r'C:\Users\lasse\Desktop\MainFolder\Final15\pdiscFinal15')

gen_model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0, beta_2=0.99, epsilon=10e-8))
disc_model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0, beta_2=0.99, epsilon=10e-8))

In [5]:
#Training set
for i in range(30000):
    img = cv2.imread(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Train\CrackC\GeneratedImage1_%s.png' % i)
    if img is None:
        noise = np.random.normal(0, 1, (1, 100))
        #label = np.random.randint(2, size=(1, 3))
        img = gen_model.predict([noise, label2])
        img = (img * 0.5) + 0.5
        #img = cv2.cvtColor(img[0], cv2.COLOR_RGB2GRAY)
        plt.imsave(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Train\CrackC\GeneratedImage1_%s.png' % i, img[0]) 

for i in range(30000):
    img = cv2.imread(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Train\FingerFailure\GeneratedImage1_%s.png' % i)
    if img is None:
        noise = np.random.normal(0, 1, (1, 100))
        #label = np.random.randint(2, size=(1, 3))
        img = gen_model.predict([noise, label1])
        img = (img * 0.5) + 0.5
        #img = cv2.cvtColor(img[0], cv2.COLOR_RGB2GRAY)
        plt.imsave(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Train\FingerFailure\GeneratedImage1_%s.png' % i, img[0])

In [6]:
#Validation set
for i in range(9000):
    img = cv2.imread(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Validation\CrackC\GeneratedImage1_%s.png' % i)
    if img is None:
        noise = np.random.normal(0, 1, (1, 100))
        #label = np.random.randint(2, size=(1, 3))
        img = gen_model.predict([noise, label2])
        img = (img * 0.5) + 0.5
        #img = cv2.cvtColor(img[0], cv2.COLOR_RGB2GRAY)
        plt.imsave(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Validation\CrackC\GeneratedImage1_%s.png' % i, img[0]) 

for i in range(9000):
    img = cv2.imread(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Validation\FingerFailure\GeneratedImage1_%s.png' % i)
    if img is None:
        noise = np.random.normal(0, 1, (1, 100))
        #label = np.random.randint(2, size=(1, 3))
        img = gen_model.predict([noise, label1])
        img = (img * 0.5) + 0.5
        #img = cv2.cvtColor(img[0], cv2.COLOR_RGB2GRAY)
        plt.imsave(r'C:\Users\lasse\Desktop\Resnets\Experiment1\Validation\FingerFailure\GeneratedImage1_%s.png' % i, img[0]) 

## Building a CNN

In [2]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(256, 256, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(3, activation='softmax')(t)
    
    model = Model(inputs, outputs)
    
    f1 = tfa.metrics.F1Score(3,'macro')
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy', f1]
    )
    return model

In [3]:
model = create_res_net() # or create_plain_net()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        batch_normalization[0][0]        
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 256, 256, 64) 0           conv2d[0][0]                     
_______________________________________________________________________________________

## Training the CNN

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=r"C:\Users\lasse\Desktop\Resnets\Experiment1\Test\\",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode=None,
    shuffle=True,
    seed=42
)

valid_generator = test_datagen.flow_from_directory(
    directory=r"C:\Users\lasse\Desktop\Resnets\Experiment1\Validation\\",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

train_generator = train_datagen.flow_from_directory(
    directory=r"C:\Users\lasse\Desktop\Resnets\Experiment1\Train\\",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 4532 images belonging to 3 classes.
Found 27065 images belonging to 3 classes.
Found 91744 images belonging to 3 classes.


In [6]:
#Line that loads dataset and labels as train set and label set.
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()

timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'SOLARPANELS-'+timestr # or 'cifar-10_plain_net_30-'+timestr

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1
)
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/'+name,
    histogram_freq=1
)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

#model.fit_generator(generator=train_generator,
#                    steps_per_epoch=STEP_SIZE_TRAIN,
#                    validation_data=valid_generator,
#                    validation_steps=STEP_SIZE_VALID,
#                    epochs=10,
#                    callbacks=[cp_callback, tensorboard_callback]
#)


model.fit(
    x=train_generator,
    epochs=100,
    verbose=1,
    validation_data=valid_generator,
    batch_size=16,
    callbacks=[cp_callback, tensorboard_callback, tf.keras.callbacks.EarlyStopping(monitor='f1_score',patience=20]
)
model.save(r'C:\Users\lasse\Desktop\ResNetExperiment')

Epoch 1/20
5734/5734 [==============================] - ETA: 0s - loss: 0.6448 - accuracy: 0.6279 - f1_score: 0.6192
Epoch 00001: saving model to checkpoints/SOLARPANELS-20210520-134521\cp-0001.ckpt
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoints/SOLARPANELS-20210520-134521\cp-0001.ckpt\assets
5734/5734 [==============================] - 2868s 500ms/step - loss: 0.6448 - accuracy: 0.6279 - f1_score: 0.6192 - val_loss: 1.1684 - val_accuracy: 0.4023 - val_f1_score: 0.3359
Epoch 2/20
5734/5734 [==============================] - ETA: 0s - loss: 0.5476 - accuracy: 0.6545 - f1_score: 0.6487
Epoch 00002: saving model to checkpoints/SOLARPANELS-20210520-134521\cp-0002.ckpt
INFO:tensorflow:Assets written to: checkpoints/SOLARPANELS-20210520-134521\cp-0002.ckpt\asset

In [10]:
model.save(r'C:\Users\lasse\Desktop\ResNetExperiment1')

INFO:tensorflow:Assets written to: C:\Users\lasse\Desktop\ResNetExperiment1\assets


## Evaluating the CNN

In [2]:
f1 = tfa.metrics.F1Score(3,'macro')

In [3]:
new_model = tf.keras.models.load_model(r'C:\Users\lasse\Desktop\ResNetExperiment', custom_objects={'F1Score': f1})
new_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[f1, 'acc'])

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=r"C:\Users\lasse\Desktop\Resnets\Experiment1\Test\\",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 4532 images belonging to 3 classes.


In [5]:
predict = new_model.evaluate_generator(test_generator, verbose=1)

Instructions for updating:
Please use Model.evaluate, which supports generators.
284/284 [==============================] - 37s 130ms/step - loss: 0.0821 - f1_score: 0.3308 - acc: 0.9846
